In [7]:
#利用生成的模拟数据进行实验

In [8]:
from typing import List

import torch
import torch.nn as nn
import argparse
import numpy as np
import time
import csv
from data import load_data
# from deep_knowledge_tracing_model import DeepKnowledgeTracing

import datetime
import os
import torch.utils.data as Data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from tensorboardX import SummaryWriter

from DKT_model import RNN

from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.metrics import r2_score

from tensorboardX import SummaryWriter

In [9]:
index=0
parser = argparse.ArgumentParser(description='DKT_LSTM model')
parser.add_argument('-epsilon', type=float, default=0.1, help='Epsilon value for Adam Optimizer')
parser.add_argument('-l2_lambda', type=float, default=0.3, help='Lambda for l2 loss')
parser.add_argument('-learning_rate', type=float, default=0.002, help='Learning rate')
parser.add_argument('-max_grad_norm', type=float, default=20, help='Clip gradients to this norm')
parser.add_argument('-keep_prob', type=float, default=0.6, help='Keep probability for dropout')
parser.add_argument('-hidden_layer_num', type=int, default=1, help='The number of hidden layers')
parser.add_argument('-concept_num', type=int, default=5, help='The number of hidden nodes')
parser.add_argument('-evaluation_interval', type=int, default=5, help='Evalutaion and print result every x epochs')
parser.add_argument('-batch_size', type=int, default=32, help='Batch size for training')
parser.add_argument('-epochs', type=int, default=100, help='Number of epochs to train')
parser.add_argument('-allow_soft_placement', type=bool, default=True, help='Allow device soft device placement')
parser.add_argument('-log_device_placement', type=bool, default=False, help='Log placement ofops on devices')
# parser.add_argument('-train_data_path', type=str, default='./遗忘_学生500题目60概念数5/答题序列数据_train'+str(index)+'.csv', help='Path to the training dataset')
# parser.add_argument('-test_data_path', type=str, default='./遗忘_学生500题目60概念数5/答题序列数据_test'+str(index)+'.csv',help='Path to the testing dataset')
# parser.add_argument('-qm_data_path', type=str, default='./遗忘_学生500题目60概念数5/题目知识点数据'+str(index)+'.csv',help='path to read q_matrix')
parser.add_argument('-train_data_path', type=str, default='./datasets/atc_datasets/s500_q10_kc5/s500_q10_kc5_train0.csv', help='Path to the training dataset')
parser.add_argument('-test_data_path', type=str, default='./datasets/atc_datasets/s500_q10_kc5/s500_q10_kc5_test0.csv',help='Path to the testing dataset')
parser.add_argument('-qm_data_path', type=str, default='./datasets/atc_datasets/s500_q10_kc5/qm_atc2.npy',help='path to read q_matrix')
parser.add_argument('-origin_DKT', type=str, default=False,help='Path to the testing dataset')
parser.add_argument('-hidden_size', type=int, default=10, help='The number of hidden nodes')

args = parser.parse_known_args()[0]

In [10]:
def run_epoch(m, optimizer, students, batch_size, num_steps, num_skills, q_matrix, state_writer, training=True, epoch=1):
    """Runs the model on the given data."""

    lr = args.learning_rate # learning rate
    train_path = args.train_data_path
    concept_num = args.concept_num
    total_loss = 0
    input_size = num_skills * 2
    start_time = time.time()
    index = 0
    actual_labels = []
    div_labels=[]
    pred_labels = []
    # hidden = m.init_hidden(batch_size)
    count = 0
#     batch_num = len(students) // batch_size
    batch_num=0
    total_auc = 0
    time_list=[]
    for i in range(num_steps):
        time_list.append(i)
#     print("数据分批次")
    while(index+batch_size < len(students)):
        batch_num+=1
        x = np.zeros((batch_size, num_steps))
        target_id: List[int] = []
        target_correctness = []
        answer_print=[]
        for i in range(batch_size):
            #student的格式：(['4'], ['77', '77', '32', '32'], ['1', '0', '0', '0'])
            student = students[index+i]
            #problem_ids:该学生答题序列的题目编号列表
            problem_ids = student[1]
            #correctness:该学生答题序列的结果列表
            correctness = student[2]
            # print("$$$$$$$$$$$$$$$len(problem_ids):",len(problem_ids))
            tmp_print = []
            for l in range(len(problem_ids)):
                tmp_print.append('('+str(problem_ids[l])+','+str(correctness[l])+')')
            answer_print.append(tmp_print)
            for j in range(len(problem_ids)-1):
                problem_id = int(problem_ids[j])
                label_index = 0
                if(int(correctness[j]) == 0):
                    label_index = problem_id
                else:
                    label_index = problem_id + num_skills
                x[i, j] = label_index
                target_id.append(i*num_steps*num_skills+j*num_skills+int(problem_ids[j+1]))
                target_correctness.append(int(correctness[j+1]))
                actual_labels.append(int(correctness[j+1]))
        index += batch_size
        count += 1
        target_id = torch.tensor(target_id, dtype=torch.int64)
        target_correctness = torch.tensor(target_correctness, dtype=torch.float)

        # One Hot encoding input data [batch_size, num_steps, input_size]
        x = torch.tensor(x, dtype=torch.int64)
        x = torch.unsqueeze(x, 2)
        input_data = torch.FloatTensor(batch_size, num_steps, input_size)
        input_data.zero_()
        input_data.scatter_(2, x, 1)
#         print("######hhm print--input_data.shape=",input_data.shape)

        target_id = target_id.cuda()
        target_correctness = target_correctness.cuda()
        if training:
            optimizer.zero_grad()
            if args.origin_DKT==True:
                output, hidden = m(input_data, hidden)
            else:
                out = m(input_data)
            stu_state = out[1]
            output = out[0]
#             print("output.shape=",output.shape)
            output = output.contiguous().view(-1)
#             print("output.shape=",output.shape)
#             print("target_id:",target_id)
            logits = torch.gather(output, 0, target_id)
#             print("logits:",logits)
            preds = torch.sigmoid(logits)
#             print("preds:",preds)
            for p in preds:
                pred_labels.append(p.item())

            criterion = nn.BCEWithLogitsLoss()
            loss = criterion(logits, target_correctness)
            
            optimizer.zero_grad()
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
#             torch.nn.utils.clip_grad_norm_(m.parameters(), args.max_grad_norm)
            optimizer.step()
            total_loss += loss.item()
        else:
            with torch.no_grad():

                
                if args.origin_DKT==True:
                    m.eval()
                    output, hidden = m(input_data, hidden)
                else:
                    out = m(input_data)
                stu_state = out[1]
                output=out[0]
#                 for i in range(output.shape[0]):
                
                for i in range(stu_state.shape[0]):
                    ans_len = len(answer_print[i])
                    state_writer.add_embedding(
                    mat=stu_state[i][:ans_len,:],#学生的能力状态向量（学生数量*hidden_size）每行表示一个数据
                    metadata=answer_print[i],#时间编号，对应时间步num_steps
                    global_step=(batch_num-1)*batch_size+i,#第几个学生
                    tag=str(epoch) #第几个轮次
                )    
                output = output.contiguous().view(-1)
                logits = torch.gather(output, 0, target_id.cuda())

                # preds
                preds = torch.sigmoid(logits)
                for p in preds:
                    pred_labels.append(p.item())

                # criterion = nn.CrossEntropyLoss()
                criterion = nn.BCEWithLogitsLoss()
                loss = criterion(logits, target_correctness)
                total_loss += loss.item()
        fpr, tpr, thresholds = metrics.roc_curve(actual_labels, pred_labels, pos_label=1)
        auc = metrics.auc(fpr, tpr)
        total_auc+=auc
        r2 = r2_score(actual_labels, pred_labels)
    average_auc = total_auc/batch_num
    average_loss = total_loss/batch_num
    print("Epoch: {},   average_AUC: {}, average_loss:{}".format(epoch,  average_auc,average_loss))
    return average_auc,average_loss,actual_labels,pred_labels

In [11]:
def main():
    train_data_path = args.train_data_path
    test_data_path  = args.test_data_path
    batch_size = args.batch_size
    concept_num = args.concept_num
    qm_data_path = args.qm_data_path
    hidden_size = args.hidden_size
    
    train_students, train_max_num_problems, train_max_skill_num = load_data(train_data_path)
    num_steps = train_max_num_problems
    num_skills = train_max_skill_num
    num_layers = 3
    test_students, test_max_num_problems, test_max_skill_num = load_data(test_data_path)

#     q_matrix = np.random.rand(num_skills,concept_num)
#     for i in range(num_skills):
#         row = q_matrix[i]
#         s = sum(row)
#         for j in range(concept_num):
#             q_matrix[i][j] = q_matrix[i][j]/s
#     q_matrix = torch.tensor(q_matrix, dtype = torch.float)

    q_matrix = np.load(qm_data_path)
#     q_matrix = np.loadtxt(qm_data_path,delimiter=',')
    q_matrix = torch.from_numpy(q_matrix)
    q_matrix = torch.tensor(q_matrix, dtype = torch.float)
    print("q_matrix.shape:",q_matrix.shape)
#     print("q_matrix:",q_matrix)
    
    print("hhm print---num_steps=",num_steps)
    print("hhm print---num_skills=",num_skills)
#     print("hhm print--q_matrix:",q_matrix)
    args.origin_DKT = False
    # model = DeepKnowledgeTracing('LSTM', num_skills*2, args.hidden_size, num_skills, num_layers)
    model = RNN(num_skills*2,hidden_size,num_skills,num_layers)
    model = torch.nn.DataParallel(model,device_ids=[0])
    model.cuda()
#     optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, eps=args.epsilon)
    optimizer=torch.optim.Adam(model.parameters(),lr=args.learning_rate)
    now=datetime.datetime.now().replace(microsecond=0) 
    time_now = (now + datetime.timedelta(hours=8))
    state_writer = SummaryWriter('stu_state/'+'DKT_e6_s500_q10_kc5'+str(num_layers)+'层')
    
    test_epoch=0
    for i in range(args.epochs):
        auc,loss,actual_labels,pred_labels = run_epoch(model, optimizer,  train_students, batch_size, num_steps, num_skills, q_matrix, state_writer, epoch=i)    
        # Testing
        if ((i + 1) % args.evaluation_interval == 0):
            auc,loss,actual_labels,pred_labels = run_epoch(model, optimizer, test_students, batch_size, num_steps, num_skills, q_matrix, state_writer, training=False, epoch=test_epoch)
            print('Testing')
            test_epoch+=1
            # print(rmse, auc, r2)

In [12]:
import warnings

warnings.filterwarnings('ignore')
main()


the number of rows is 1200
The number of students is  400
Finish reading data
the number of rows is 300
The number of students is  100
Finish reading data
q_matrix.shape: torch.Size([10, 5])
hhm print---num_steps= 27
hhm print---num_skills= 10
Epoch: 0,   average_AUC: 0.5842508194982926, average_loss:0.6990961382786433
Epoch: 1,   average_AUC: 0.618450734841445, average_loss:0.6848550140857697
Epoch: 2,   average_AUC: 0.6529115860345732, average_loss:0.6634921729564667
Epoch: 3,   average_AUC: 0.6806056437806355, average_loss:0.6300158500671387
Epoch: 4,   average_AUC: 0.6795801408277891, average_loss:0.6008747766415278
Epoch: 0,   average_AUC: 0.6402934133699076, average_loss:0.6133372187614441
Testing
Epoch: 5,   average_AUC: 0.6758751630018714, average_loss:0.5857284814119339
Epoch: 6,   average_AUC: 0.6791199630563926, average_loss:0.5790772835413615
Epoch: 7,   average_AUC: 0.6866154564115506, average_loss:0.5757156610488892
Epoch: 8,   average_AUC: 0.6907062652454566, average_los